In [2]:
import pandas as pd
import random

import pandas as pd

# Specify the file path to your CSV file here
csv_file_path = ('/Users/geethika/Downloads/Sample Data/Sample_selection.csv')

# Step 1: Read CSV data and validate it
def read_csv(file_path):
    try:
        data = pd.read_csv(file_path)
        if set(["alphabet", "alphabet_team", "alphabet_category", "selection_percentage"]).issubset(data.columns):
            return data
        else:
            raise Exception("Invalid CSV format. Required columns are missing.")
    except Exception as e:
        print(f"Error: {e}")
        return None

data = read_csv(csv_file_path)
if data is None:
    exit()


# Step 2: Group the alphabet data by category
category_groups = data.groupby("alphabet_category")

# Step 3: Generate Subsets
subsets = []

for category, group in category_groups:
    group = group.sample(frac=1)  # Shuffle the group for randomization

   # ...
while len(group) >= 3:
    subset = group[group["alphabet_category"] == category].head(1)
    group = group[group["alphabet_category"] != category]

    while len(subset) < 8 and not group.empty:
        # Check if there are any alphabets left in the category to sample from.
        if (group["alphabet_category"] == category).any():
            alphabet = group[group["alphabet_category"] == category].sample(1, weights=group["selection_percentage"].str.rstrip('%').astype(float))
            subset = pd.concat([subset, alphabet], ignore_index=True)
            group = group.drop(alphabet.index)
        else:
            break  # No more alphabets in the category, exit the loop.

    other_categories = set(category_groups.groups.keys()) - {category}
    for other_category in other_categories:
        # Check if there are any alphabets left in the other category to sample from.
        if (group["alphabet_category"] == other_category).any():
            alphabet = group[group["alphabet_category"] == other_category].sample(1)
            subset = pd.concat([subset, alphabet], ignore_index=True)
            group = group.drop(alphabet.index)

# Step 4: Validate Subsets
def validate_subsets(subsets):
    for subset in subsets:
        categories = set(subset["alphabet_category"])
        if len(categories) != len(category_groups.groups):
            return False
        if len(subset) != 11:
            return False
    return True

if validate_subsets(subsets):
    print("Subsets are valid.")
else:
    print("Error: Subsets do not meet the criteria.")

# Step 5: Save Subsets to CSV files
for i, subset in enumerate(subsets):
    subset.to_csv(f"subset_{i+1}.csv", index=False)


Subsets are valid.
